In [1]:
import sqlite3
import autogen
import requests
import json
import pdb
import os
import re
import tiktoken


from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

from termcolor import colored
import random

In [2]:
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

gpt4_llm_config = {"config_list": config_list_gpt4, "cache_seed": 42}



config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-35-turbo", "gpt-3.5-turbo"],
    },
)

gpt35_llm_config = {"config_list": config_list_gpt35, "cache_seed": 42}

In [3]:
import requests
import zipfile
import io

# URL of the file to be downloaded
url = "https://azurecloudpublicdataset2.blob.core.windows.net/azurepublicdatasetv2/azurevmallocation_dataset2020/AzurePackingTraceV1.zip"
folder_name = "vm"
filename = os.path.join(folder_name, "packing_trace_zone_a_v1.sqlite")

if not os.path.exists(filename):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create a ZipFile object in memory using the content of the response
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Extract all the contents into a desired directory
            z.extractall(folder_name)
        print("File downloaded and unzipped successfully.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
    
# filename = "Chinook_Sqlite.sqlite"
filename = os.path.join(folder_name, "packing_trace_zone_a_v1.sqlite")
assert os.path.exists(filename)

File downloaded and unzipped successfully.


In [23]:
# From https://github.com/microsoft/autogen/blob/9888f7abb11292caf1170803306774eff40b8306/notebook/agentchat_sql.ipynb


# Function to list all tables in the database
def list_tables(cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    ans = []
    for table in cursor.fetchall():
        ans.append(table[0])
    return ans
    
# Function to show the schema of a table
def show_table_schema(cursor, table_name):
    cursor.execute(f"PRAGMA table_info({table_name});")
    
    ans = f"Here is the table information for Table: {table_name}\n"
    ans += "-" * 30 + "\n"
    ans += "Column ID | Column Name | Data Type | Not Null Constraint | Default Val | Primary Key | \n"
    ans += "-" * 30 + "\n"
    for row in cursor.fetchall():
        ans += " | ".join([str(x) for x in row]) + "\n"
    ans += "-" * 30 + "\n\n\n"
    return ans

# Function to execute a query
def execute_query(cursor, query):
    cursor.execute(query)
    return cursor.fetchall()


def find_potential_joins(cursor):
    # Query to get foreign key relationships
    cursor.execute("SELECT tbl_name, sql FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()

    joins = []
    for table, create_statement in tables:
        cursor.execute(f"PRAGMA foreign_key_list({table})")
        foreign_keys = cursor.fetchall()
        for fk in foreign_keys:
            # Each item in foreign_keys is a tuple like:
            # (id, seq, table, from, to, on_update, on_delete, match)
            from_table = table
            from_column = fk[3]
            to_table = fk[2]
            to_column = fk[4]
            join_info = f"{from_table}.{from_column} -> {to_table}.{to_column}"
            joins.append(join_info)

    return joins

def column_names(cursor, table_name):
    """
    Retrieves the column names for a specified table.
    
    Args:
    cursor (sqlite3.Cursor): The cursor object connected to the database.
    table_name (str): The name of the table for which to retrieve column names.

    Returns:
    list: A list of column names for the specified table.
    """
    query = f"PRAGMA table_info({table_name});"
    cursor.execute(query)
    # Fetches all rows from the query result, extracts the second column which contains column names
    return [row[1] for row in cursor.fetchall()]

def table_prompt(cursor, question="", model="gpt-4", max_tokens=5000):
    db_info = ""
    if question:
        db_info = f"## My question\n{question}\n\n\n"
        
    db_info += "## Table information\n"
    tables = list_tables(cursor)
    db_info += f"There are {len(tables)} tables in the database, which are:"
    for t in tables:
        db_info += "\n- " + t
        
    joins = find_potential_joins(cursor)
    if joins:
        db_info += "\n\n## Join information\n"
        db_info += "Here are the columns (and their tables) that could be potentially joined:"
        for j in joins:
            db_info += "\n- " + j
        db_info += "\n\n"    
        
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    n_tokens = lambda x: len(encoding.encode(x))
    if max_tokens is None:
        max_tokens = 10e9
        
    db_info = db_info
    assert n_tokens(db_info) < max_tokens
    
    #### First, try full schema info
    schema_info = "\n\n## Schema\n"
    for table in tables:
        table_info = show_table_schema(cursor, table)
        schema_info += table_info

    if n_tokens(schema_info + schema_info) < max_tokens:
        return db_info + schema_info
        
    #### if not possible, try column names
    column_info = "\n\n## Columns\n"
    for table in tables:
        column_info += f"{table}: {column_names(cursor, table)}\n"

    if n_tokens(column_info + column_info) < max_tokens:
        return db_info + column_info
    
    alert = "\n\n### Schemas are Skipped for some tables.\n__Note__: "
    alert += "You SHOULD use the `column_names(cursor, table_name)` function to show columns of a table."
    
    return db_info + alert


def execute_code_with_cursor(code, cursor, **kwargs) -> Dict:
    locals_dict = {"cursor": cursor, "show_table_schema": show_table_schema, "column_names": column_names}

    if 'lang' in kwargs and kwargs['lang'].lower() == "sql":
        print(colored("WARNING: raw sql code is received for execution. We allow it for now, but the LLM should be improved.", "yellow"))
        code = f"rst = cursor.execute({code}).fetchall()"

    try:
        exec(code, locals_dict, locals_dict)
    except Exception as e:
        return "Error: " + str(e)
    if "rst" in locals_dict:
        return {"rst": locals_dict["rst"]}
    known_keys = list(locals().keys()) + list(globals().keys())
    return {k: v for k, v in locals_dict.items() if k not in known_keys}
    

In [6]:
connection = sqlite3.connect(filename)
print(table_prompt(connection.cursor(), max_tokens=100))


## Table information
There are 2 tables in the database, which are:
- vm
- vmType

### Schemas are Skipped for some tables.
__Note__: You SHOULD use the `column_names(cursor, table_name)` function to show columns of a table.


In [46]:
column_names(connection.cursor(),"vm")

['vmId', 'tenantId', 'vmTypeId', 'priority', 'starttime', 'endtime']

In [10]:
column_names(connection.cursor(),"vmType")

['id', 'vmTypeId', 'machineId', 'core', 'memory', 'hdd', 'ssd', 'nic']

In [6]:
# From https://github.com/microsoft/autogen/blob/9888f7abb11292caf1170803306774eff40b8306/notebook/agentchat_sql.ipynb
termination_notice = (
    '\n\nDo not show appreciation in your responses, say only what is necessary. '
    'if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE '
    'to indicate the conversation is finished and this is your last message.'
)
DEFAULT_SQL_CODE_CONFIG = {
                "last_n_messages": 3,
                "work_dir": ".",
                "use_docker": False
            }

def IS_TERMINATE_MSG_FOR_SQL_ASSISTANT(x): 
    content = content_str(x.get("content"))
    if content is None:
        return True;
    return content.find("ANSWER") >= 0 or content is None
    
    
def IS_TERMINATE_MSG_FOR_SQL_PROXY(x): 
    content = content_str(x.get("content"))
    return content.find("ANSWER") >= 0 and re.findall(r"```.+```", content, re.DOTALL) == []

class SQLAgent(ConversableAgent):
    
    def __init__(self, sql_file, name, max_sys_msg_tokens, llm_config: dict, 
                 **kwargs):
        super().__init__(name, llm_config=llm_config, max_consecutive_auto_reply=0,
                         **kwargs)
        self.sql_file = sql_file
        connection = sqlite3.connect(sql_file)
        self.cursor = connection.cursor()
        
        self.assistant = AssistantAgent(name="sql llm", system_message="""You can interact with the SQLlite3 database and answer questions.
        
There are two types of answers you can provide: code and answer.

Do not show appreciation in your responses, say only what is necessary. 
if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE 
'to indicate the conversation is finished and this is your last message.
                                        
## Code
In the code format, you must write code directly in code block to answer questions. For instance,
```python
# Python code by using the `cursor` variable.
```
The cursor variable is provided directly to you. 
Save the result to the "rst" variable so that you can see the output.
Note that you are interacting with a Python environment, and it can only reply you if you write code.


## Answer
If you gather enough information (e.g., from coding or prior knowledge) to answer the user question, then you should reply with your final answer in the foramt:
ANSWER: your answer goes here.
""" + table_prompt(self.cursor, question="", model=llm_config["config_list"][0]["model"], max_tokens=max_sys_msg_tokens), 
                                        llm_config=llm_config,
                                        is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_ASSISTANT,
                                       max_consecutive_auto_reply=10)
        
        self.proxy = SQLProxy(sql_file=filename, name="sql proxy", 
                              human_input_mode="NEVER",
                             is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_PROXY,   
                             code_execution_config={
        "work_dir": "../autogen_docker/",
        "use_docker": "autogen_base_img"
    },)
        self.register_reply([Agent, None], SQLAgent.generate_sql_reply)
        self.synth_history()
        self.cursor.close()
        connection.close()
        
    def synth_history(self):
        # we inject some synthetic Q&A with the desired format, so that the LLM can learn the pattern in context.
        tables = list_tables(self.cursor)
        n_tables = len(tables)
        if(n_tables ==0):
            prior=[]
        else:
                
            query = f"SELECT COUNT(*) FROM {tables[0]};"
            n_rows_table0 = self.cursor.execute(query).fetchall()[0][0]        
            table0_info = show_table_schema(self.cursor, tables[0])
            self.table_memory = {table_name: show_table_schema(self.cursor, table_name) for table_name in tables}
            
            prior = [{'content': 'QUESTION: How many tables are in the database?',
    'role': 'user'},
    {'content': 'We can count the number of tables in the database.\n```python\ncursor.execute("SELECT COUNT(name) FROM sqlite_master WHERE type=\'table\';")\nrst = cursor.fetchone()\nrst\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': (" + str(n_tables) + ",)}",
    'role': 'user'},
    {'content': f'ANSWER: The database contains {n_tables} tables. If you need any more information, feel free to ask!',
    'role': 'assistant'},
    {'content': f'QUESTION: How many rows are in the {tables[0].lower()} table?',
    'role': 'user'},
    {'content': f'We can run the following code\n```python\nquery = "SELECT COUNT(*) FROM {tables[0]};"\nrst = cursor.execute(query).fetchall()\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': (" + str(n_rows_table0) + ",)}",
    'role': 'user'},
    {'content': f'ANSWER: There are {n_rows_table0} rows in the {tables[0]} table.',
    'role': 'assistant'},
    {'content': f'QUESTION: What columns are in the {tables[0].lower()} table?',
    'role': 'user'},
    {'content': f'We can run the function\n```python\nrst = show_table_schema(cursor, {tables[0]})\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': " + str(table0_info) + "}",
    'role': 'user'},
    {'content': f'ANSWER: The columns in the {tables[0]} table are: ' + ", ".join(column_names(self.cursor, tables[0])),
    'role': 'assistant'},
    {'content': "QUESTION: Who are you?",
    'role': 'user'},
    {'content': "ANSWER: Sorry, I don't have enough information to answer your question, because it is not in the database.",
    'role': 'assistant'},
    ]
        self.assistant._oai_messages[self.proxy] = prior
        
    def generate_sql_reply(self, messages: Optional[List[Dict]], sender: "Agent", config):
        """Generate a reply using OpenAI DALLE call."""
        client = self.client if config is None else config
        if client is None:
            return False, None
        if messages is None:
            messages = self._oai_messages[sender]
        prompt = messages[-1]["content"]

        self.proxy.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        ans = content_str(self.assistant.last_message()["content"])
        ans = ans.replace("ANSWER:", "").strip().rstrip()
        
        return True, ans
    
class SQLProxy(UserProxyAgent):
    def __init__(self, sql_file, name, 
                 code_execution_config: dict = DEFAULT_SQL_CODE_CONFIG, **kwargs):
        super().__init__(name,
                         code_execution_config=code_execution_config, **kwargs)
        self.sql_file = sql_file

        
    def run_code(self, code, **kwargs):
        """Run the code and return the result.

        Override this function to modify the way to run the code.
        Args:
            code (str): the code to be executed.
            **kwargs: other keyword arguments.

        Returns:
            A tuple of (exitcode, logs, image).
            exitcode (int): the exit code of the code execution.
            logs (str): the logs of the code execution.
            image (str or None): the docker image used for the code execution.
        """
        connection = sqlite3.connect(self.sql_file)
        self.cursor = connection.cursor()
        rst = execute_code_with_cursor(code, self.cursor, **kwargs)
        exitcode = 0 if str(rst).find("Error: ") != 0 else 1    
        connection.commit()
        self.cursor.close()
        connection.close()
        
        return exitcode, str(rst), None
        


In [7]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="NEVER", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="Which VM has the longest runtime and what type is it?")

user (to SQL Helper):

Which VM has the longest runtime and what type is it?

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: Which VM has the longest runtime and what type is it?

--------------------------------------------------------------------------------


/Users/ciuji/repo-workspace/autogen/pyautogen/lib/python3.11/site-packages/autogen/agentchat/user_proxy_agent.py:82: UserWarning: Using None to signal a default code_execution_config is deprecated. Use {} to use default or False to disable code execution.
  super().__init__(


sql llm (to sql proxy):

To find the VM with the longest runtime and its type, we can use the following query:
```python
query = "SELECT v.vmId, t.vmTypeId FROM vm AS v JOIN vmType AS t ON v.vmTypeId = t.vmTypeId ORDER BY v.endtime - v.starttime DESC LIMIT 1;"
rst = cursor.execute(query).fetchone()
rst
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
sql proxy (to sql llm):

exitcode: 0 (execution succeeded)
Code output: 
{'rst': (9543, 41)}

--------------------------------------------------------------------------------
sql llm (to sql proxy):

ANSWER: The VM with the longest runtime is vmId 9543 and its type is vmTypeId 41.

--------------------------------------------------------------------------------
SQL Helper (to user):

The VM with the longest runtime is vmId 9543 and its type is vmTypeId 41.

--------------------------------------------------------------------------------


In [8]:
print(sql_assistant.assistant.system_message)
print(sql_assistant._oai_messages)

You can interact with the SQLlite3 database and answer questions.
        
There are two types of answers you can provide: code and answer.

Do not show appreciation in your responses, say only what is necessary. 
if "Thank you" or "You're welcome" are said in the conversation, then say TERMINATE 
'to indicate the conversation is finished and this is your last message.
                                        
## Code
In the code format, you must write code directly in code block to answer questions. For instance,
```python
# Python code by using the `cursor` variable.
```
The cursor variable is provided directly to you. 
Save the result to the "rst" variable so that you can see the output.
Note that you are interacting with a Python environment, and it can only reply you if you write code.


## Answer
If you gather enough information (e.g., from coding or prior knowledge) to answer the user question, then you should reply with your final answer in the foramt:
ANSWER: your answer goes h

In [10]:
user.initiate_chat(sql_assistant, message="How many VM do I Have")

user (to SQL Helper):

How many VM do I Have

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: How many VM do I Have

--------------------------------------------------------------------------------
sql llm (to sql proxy):

We can count the number of VMs you have by running the following code:
```python
query = "SELECT COUNT(DISTINCT vmId) FROM vm;"
rst = cursor.execute(query).fetchone()
rst
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
sql proxy (to sql llm):

exitcode: 0 (execution succeeded)
Code output: 
{'rst': (5559800,)}

--------------------------------------------------------------------------------
sql llm (to sql proxy):

ANSWER: You have a total of 5559800 VMs.

--------------------------------------------------------------------------------
SQL Helper (to user):

You have a total of 5559800 VMs.

------

## Modifying SQL Agent

In [125]:
# From https://github.com/microsoft/autogen/blob/9888f7abb11292caf1170803306774eff40b8306/notebook/agentchat_sql.ipynb
termination_notice = (
    '\n\nDo not show appreciation in your responses, say only what is necessary. '
    'if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE '
    'to indicate the conversation is finished and this is your last message.'
)
DEFAULT_SQL_CODE_CONFIG = {
                "last_n_messages": 3,
                "work_dir": "../autogen_docker/",
                "use_docker": "autogen_base_img"
            }

def IS_TERMINATE_MSG_FOR_SQL_ASSISTANT(x): 
    content = content_str(x.get("content"))
    if content is None or content == "":
        return True
    return content.find("ANSWER") >= 0 or content.find("ACCEPT") >=0
    
    
def IS_TERMINATE_MSG_FOR_SQL_PROXY(x): 
    content = content_str(x.get("content"))
    return content.find("ANSWER") >= 0 and re.findall(r"```.+```", content, re.DOTALL) == []

class SQLGuard(ConversableAgent):
    def __init__(self, sql_file, name, max_sys_msg_tokens, llm_config: dict, 
                 **kwargs):
        super().__init__(name, llm_config=llm_config, max_consecutive_auto_reply=0,
                         **kwargs)
        self.sql_file = sql_file
        self.guard = AssistantAgent(name="sql guard",system_message="""
You are interact with the SQLlite3 database and review the code to operating database, you should detect some dangerous operation like deleting tables, deleting databases, creating databases. If you think the operation is dangerous, reject it, and output `REJECT` in your reply. If you think the operation is safe, accept it, reply `ACCEPT` plus the original code.""",llm_config=llm_config,
                                        is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_ASSISTANT,
                                       max_consecutive_auto_reply=2)
        self.register_reply([Agent, None], SQLAgent.generate_guard_reply)
    
    def generate_guard_reply(self, messages: Optional[List[Dict]], sender: "Agent", config):
        """Generate a reply using OpenAI DALLE call."""
        client = self.client if config is None else config
        if client is None:
            return False, None
        if messages is None:
            messages = self._oai_messages[sender]
        prompt = messages[-1]["content"]

        self.guard.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        # self.proxy.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        print("guard feedback",self.guard.last_message()['content'])

        ans = content_str(self.assistant.last_message(self.guard)["content"])
        ans = ans.replace("ANSWER:", "").strip().rstrip()
        print("receiving from ",sender)
        print("reply ",ans)
        
        return True, ans

class SQLAgent(ConversableAgent):
    
    def __init__(self, sql_file, name, max_sys_msg_tokens, llm_config: dict, 
                 **kwargs):
        super().__init__(name, llm_config=llm_config, max_consecutive_auto_reply=0,
                         **kwargs)
        self.sql_file = sql_file
        connection = sqlite3.connect(sql_file)
        self.cursor = connection.cursor()
        
        self.assistant = AssistantAgent(name="sql llm", system_message="""You can interact with the SQLlite3 database and answer questions.
        
There are two types of answers you can provide: code and answer.

Do not show appreciation in your responses, say only what is necessary. 
if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE 
'to indicate the conversation is finished and this is your last message.
                                        
## Code
In the code format, you must write code directly in code block to answer questions. For instance,
```python
# Python code by using the `cursor` variable.
```
The cursor variable is provided directly to you. 
Save the result to the "rst" variable so that you can see the output.
Note that you are interacting with a Python environment, and it can only reply you if you write code.


## Answer
If you gather enough information (e.g., from coding or prior knowledge) to answer the user question, then you should reply with your final answer in the foramt:
ANSWER: your answer goes here.
""" + table_prompt(self.cursor, question="", model=llm_config["config_list"][0]["model"], max_tokens=max_sys_msg_tokens), 
                                        llm_config=llm_config,
                                        is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_ASSISTANT,
                                       max_consecutive_auto_reply=5)
        
        self.guard = AssistantAgent(name="sql guard",system_message="""
You are interact with the SQLlite3 database and review the code before running it, you should detect some unexpectedly operation like deleting tables, deleting databases, creating databases or example sql cammands. 
If the message you receive is juat an example code, reject it by replying `REJECT`.  
Reject unconsciously example code. 
If you think the operation is safe, accept it, reply `ACCEPT` PLUS the original message you receive.
Don't forget to paste the message you receive as reply if you accept the operation. 
                                    """,llm_config=llm_config,
                                        is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_ASSISTANT,
                                       max_consecutive_auto_reply=1)
        
        self.proxy = SQLProxy(sql_file=filename, name="sql proxy", 
                              human_input_mode="NEVER",
                             is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_PROXY,   
                             code_execution_config={
        "work_dir": "../autogen_docker/",
        "use_docker": "autogen_base_img"
    },)
        self.register_reply([Agent, None], SQLAgent.generate_sql_reply)
        self.assistant.register_reply([self.proxy, None], SQLAgent.assistant_reply_proxy)
        self.guard.register_reply([self.proxy, None], SQLAgent.assistant_reply_proxy)

        self.synth_history()
        self.cursor.close()
        connection.close()
    
    def assistant_reply_proxy(self, messages: Optional[List[Dict]], sender: "Agent", config):
        return False,None
        

        

    def synth_history(self):
        # we inject some synthetic Q&A with the desired format, so that the LLM can learn the pattern in context.
        tables = list_tables(self.cursor)
        n_tables = len(tables)
        if(n_tables ==0):
            prior=[]
        else:
                
            query = f"SELECT COUNT(*) FROM {tables[0]};"
            n_rows_table0 = self.cursor.execute(query).fetchall()[0][0]        
            table0_info = show_table_schema(self.cursor, tables[0])
            self.table_memory = {table_name: show_table_schema(self.cursor, table_name) for table_name in tables}
            
            prior = [{'content': 'QUESTION: How many tables are in the database?',
    'role': 'user'},
    {'content': 'We can count the number of tables in the database.\n```python\ncursor.execute("SELECT COUNT(name) FROM sqlite_master WHERE type=\'table\';")\nrst = cursor.fetchone()\nrst\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': (" + str(n_tables) + ",)}",
    'role': 'user'},
    {'content': f'ANSWER: The database contains {n_tables} tables. If you need any more information, feel free to ask!',
    'role': 'assistant'},
    {'content': f'QUESTION: How many rows are in the {tables[0].lower()} table?',
    'role': 'user'},
    {'content': f'We can run the following code\n```python\nquery = "SELECT COUNT(*) FROM {tables[0]};"\nrst = cursor.execute(query).fetchall()\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': (" + str(n_rows_table0) + ",)}",
    'role': 'user'},
    {'content': f'ANSWER: There are {n_rows_table0} rows in the {tables[0]} table.',
    'role': 'assistant'},
    {'content': f'QUESTION: What columns are in the {tables[0].lower()} table?',
    'role': 'user'},
    {'content': f'We can run the function\n```python\nrst = show_table_schema(cursor, {tables[0]})\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': " + str(table0_info) + "}",
    'role': 'user'},
    {'content': f'ANSWER: The columns in the {tables[0]} table are: ' + ", ".join(column_names(self.cursor, tables[0])),
    'role': 'assistant'},
    {'content': "QUESTION: Who are you?",
    'role': 'user'},
    {'content': "ANSWER: Sorry, I don't have enough information to answer your question, because it is not in the database.",
    'role': 'assistant'},
    ]
        self.assistant._oai_messages[self.proxy] = prior
    
    def generate_assistant_reply_proxy(self, messages: Optional[List[Dict]], sender: "Agent", config):
        """Generate a reply using OpenAI DALLE call."""
        client = self.client if config is None else config
        if client is None:
            return False, None
        if messages is None:
            messages = self._oai_messages[sender]
        prompt = messages[-1]["content"]

        self.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        # self.proxy.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        if self._oai_messages[self.assistant][-1]['role']=="assistant":
            guard_reply = self._oai_messages[self.assistant][-1]['content']
            if("ACCEPT" in guard_reply):
                print("Accept prompt")
                self.initiate_chat(self.proxy, message=guard_reply, clear_history=False)

        # print("guard feedback",self.guard._oai_messages[self.assistant][-1]['content'])

        ans = content_str(self.assistant.last_message(self.proxy)["content"])
        ans = ans.replace("ANSWER:", "").strip().rstrip()
        # print("receiving from ",sender)
        # print("reply ",ans)
        
        return True, ans
        
    def generate_sql_reply(self, messages: Optional[List[Dict]], sender: "Agent", config):
        """Generate a reply using OpenAI DALLE call."""
        client = self.client if config is None else config
        if client is None:
            return False, None
        if messages is None:
            messages = self._oai_messages[sender]
        prompt = messages[-1]["content"]

        self.guard.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        # self.proxy.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        if self.guard._oai_messages[self.assistant][-1]['role']=="assistant":
            guard_reply = self.guard._oai_messages[self.assistant][-1]['content']
            if("ACCEPT" in guard_reply):
                print("Accept prompt")
                self.assistant.initiate_chat(self.proxy, message=guard_reply, clear_history=False)

        # print("guard feedback",self.guard._oai_messages[self.assistant][-1]['content'])

        ans = content_str(self.assistant.last_message(self.proxy)["content"])
        ans = ans.replace("ANSWER:", "").strip().rstrip()
        # print("receiving from ",sender)
        # print("reply ",ans)
        
        return True, ans
    
class SQLProxy(UserProxyAgent):
    def __init__(self, sql_file, name, 
                 code_execution_config: dict = DEFAULT_SQL_CODE_CONFIG, **kwargs):
        super().__init__(name,
                         code_execution_config=code_execution_config, **kwargs)
        self.sql_file = sql_file

        
    def run_code(self, code, **kwargs):
        """Run the code and return the result.

        Override this function to modify the way to run the code.
        Args:
            code (str): the code to be executed.
            **kwargs: other keyword arguments.

        Returns:
            A tuple of (exitcode, logs, image).
            exitcode (int): the exit code of the code execution.
            logs (str): the logs of the code execution.
            image (str or None): the docker image used for the code execution.
        """
        connection = sqlite3.connect(self.sql_file)
        self.cursor = connection.cursor()
        rst = execute_code_with_cursor(code, self.cursor, **kwargs)
        exitcode = 0 if str(rst).find("Error: ") != 0 else 1    
        connection.commit()
        self.cursor.close()
        connection.close()
        
        return exitcode, str(rst), None
        


In [118]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="NEVER", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="How many vm do I have?")

user (to SQL Helper):

How many vm do I have?

--------------------------------------------------------------------------------
sql guard (to sql llm):

QUESTION: How many vm do I have?

--------------------------------------------------------------------------------


/Users/ciuji/repo-workspace/autogen/pyautogen/lib/python3.11/site-packages/autogen/agentchat/user_proxy_agent.py:82: UserWarning: Using None to signal a default code_execution_config is deprecated. Use {} to use default or False to disable code execution.
  super().__init__(


sql llm (to sql guard):

```python
# Count the number of rows in the 'vm' table
cursor.execute("SELECT COUNT(*) FROM vm")
rst = cursor.fetchone()[0]
rst
```

--------------------------------------------------------------------------------
sql guard (to sql llm):

ACCEPT
```python
# Count the number of rows in the 'vm' table
cursor.execute("SELECT COUNT(*) FROM vm")
rst = cursor.fetchone()[0]
rst
```

--------------------------------------------------------------------------------
Accept prompt
sql llm (to sql proxy):

ACCEPT
```python
# Count the number of rows in the 'vm' table
cursor.execute("SELECT COUNT(*) FROM vm")
rst = cursor.fetchone()[0]
rst
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
sql proxy (to sql llm):

exitcode: 0 (execution succeeded)
Code output: 
{'rst': 5559800}

--------------------------------------------------------------------------------
sql llm (to sql p

In [22]:
sql_assistant._oai_messages[user][-1]

{'content': "exitcode: 1 (execution failed)\nCode output: \nError: name 'vmId' is not defined",
 'role': 'assistant'}

In [83]:
sql_assistant.guard.chat_messages

defaultdict(list,
            {<autogen.agentchat.assistant_agent.AssistantAgent at 0x28fcdf7d0>: [{'content': 'QUESTION: How many vm do I have?',
               'role': 'assistant'},
              {'content': '```python\n# Count the number of rows in the \'vm\' table\ncursor.execute("SELECT COUNT(*) FROM vm")\nrst = cursor.fetchone()[0]\nrst\n```',
               'role': 'user'},
              {'content': 'ACCEPT\n\n```python\n# Count the number of rows in the \'vm\' table\ncursor.execute("SELECT COUNT(*) FROM vm")\nrst = cursor.fetchone()[0]\nrst\n```',
               'role': 'assistant'}]})

In [81]:
sql_assistant.assistant._oai_messages[sql_assistant.guard][-1]['role']

'user'

In [68]:
sql_assistant.guard._oai_messages[sql_assistant.assistant]

[{'content': 'QUESTION: How many vm do I have?', 'role': 'assistant'},
 {'content': '```python\n# Count the number of rows in the \'vm\' table\ncursor.execute("SELECT COUNT(*) FROM vm")\nrst = cursor.fetchone()[0]\nrst\n```',
  'role': 'user'},
 {'content': 'ACCEPT\n\n```python\n# Count the number of rows in the \'vm\' table\ncursor.execute("SELECT COUNT(*) FROM vm")\nrst = cursor.fetchone()[0]\nrst\n```',
  'role': 'assistant'},
 {'content': 'ANSWER: You have 20 vm.', 'role': 'user'}]

In [126]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="ALWAYS", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="how can i insert an data entry to the db")

user (to SQL Helper):

how can i insert an data entry to the db

--------------------------------------------------------------------------------
sql guard (to sql llm):

QUESTION: how can i insert an data entry to the db

--------------------------------------------------------------------------------
sql llm (to sql guard):

To insert a data entry into the database, you need to execute an SQL INSERT statement using the cursor object. Here is an example of how to insert a new entry into the 'vm' table:

```python
# Connect to the database
import sqlite3
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Define the data for the new entry
vmId = 1
tenantId = 1
vmTypeId = 1
priority = 5
starttime = '2021-01-01 00:00:00'
endtime = '2021-01-02 00:00:00'

# Execute the INSERT statement
cursor.execute("INSERT INTO vm (vmId, tenantId, vmTypeId, priority, starttime, endtime) VALUES (?, ?, ?, ?, ?, ?)", (vmId, tenantId, vmTypeId, priority, starttime, endtime))

# Commit the tr

In [128]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="NEVER", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="how can i delete the database")

user (to SQL Helper):

how can i delete the database

--------------------------------------------------------------------------------
sql guard (to sql llm):

QUESTION: how can i delete the database

--------------------------------------------------------------------------------
sql llm (to sql guard):

To delete the SQLite database, you can simply delete the file containing the database. 

If you are using a filesystem, you can delete the file using the operating system's file management functions or commands. 

If you are using a programming language, you can use the corresponding file deletion function or method to delete the file. For example, in Python, you can use the `os.remove()` function to delete the file.

Please note that deleting the database file will permanently remove all the data stored in it. Make sure to backup your data if you need it before deleting the database.

--------------------------------------------------------------------------------
sql guard (to sql l